In [1]:
# Imports and Consts
SEED = 42
import struct as st
import numpy as np

In [2]:
# Helper function to parse IDX files
def parse_idx(file_path):
    with open(file_path, 'rb') as file:
        magic = st.unpack('>I', file.read(4))[0]  # Magic number (4 bytes)
        num_items = st.unpack('>I', file.read(4))[0]  # Number of items (4 bytes)

        if magic == 2051:  # Magic number for images
            num_rows = st.unpack('>I', file.read(4))[0]
            num_cols = st.unpack('>I', file.read(4))[0]
            num_bytes = num_items * num_rows * num_cols
            data = np.frombuffer(file.read(num_bytes), dtype=np.uint8)
            return data.reshape(num_items, num_rows, num_cols)
        elif magic == 2049:  # Magic number for labels
            data = np.frombuffer(file.read(num_items), dtype=np.uint8)
            return data
        else:
            raise ValueError(f"Unknown magic number: {magic}")

# Parse the training data
x_train = parse_idx('DigitData/train-images.idx3-ubyte')
y_train = parse_idx('DigitData/train-labels.idx1-ubyte')

x_test = parse_idx('DigitData/t10k-images.idx3-ubyte')
y_test = parse_idx('DigitData/t10k-labels.idx1-ubyte')

# Reshape and scale down
x_train = x_train.reshape(x_train.shape[0], -1) / 255.0
x_test = x_test.reshape(x_test.shape[0], -1) / 255.0

In [13]:
# Normalize
train_mean = np.mean(x_train, axis=0)
train_std = np.std(x_train, axis=0) + 1e-12

x_train = (x_train - train_mean) / train_std
x_test = (x_test - train_mean) / train_std

# Add extra ones column
x_train = np.hstack([x_train, np.ones((len(x_train), 1)) ])
x_test = np.hstack([ x_test, np.ones((len(x_test), 1)) ])

In [14]:
# Helpers

def Pr(arr: np.ndarray, param: np.ndarray) -> np.ndarray:
    z = arr @ param
    z_clipped = np.clip(z, -500, +500)
    return 1.0 / (1.0 + np.exp(-z_clipped))

def construct_w(p: np.ndarray) -> np.ndarray:
    return (1 - p) * p

def compute_delta(X: np.ndarray, w: np.ndarray, Y: np.ndarray, p: np.ndarray) -> np.ndarray:
    XL = X * w[:, None]
    H = XL.T @ X + 1e-2 * np.eye(X.shape[1])
    B = X.T @ (Y-p)

    # Speed up computation via decomposition
    L = np.linalg.cholesky(H)
    
    # Solve first only with L
    temp = np.linalg.solve(L, B)

    # Return final solution by solving for temp with L.T
    return np.linalg.solve(L.T, temp)

# MIGHT REMOVE

# def loglik(p, y, tol=1e-15):
#     ll = np.sum(y * np.log(p + tol) + (1 - y) * (np.log(1 - p + tol)))
#     return ll

# def scale_const(X, Y, p_old, theta, delta, alpha):
#     ll_old = loglik(p_old, Y)

#     while True:
#         p_new = Pr(X, theta + alpha * delta)
#         ll_new = loglik(p_new, Y)

#         if ll_new >= ll_old:
#             return alpha
        
#         if alpha < 1e-8:
#             return None
        
#         alpha *= 0.5

In [18]:
# Model Training

def newton_method(X, Y, theta=None, iters=1000):
    # Initialize Theta
    np.random.seed(42)

    if theta is None:
        theta = np.random.randn(X.shape[1])

    for _ in range(iters):
        # Calc Values
        p = Pr(X, theta)
        w = construct_w(p)
        delta = compute_delta(X, w, Y, p)

        # Apply Newton method
        theta += delta
    return theta


def multi_newton_method(X, Y, thetas=None, iters=25):
    if thetas is None:
        thetas = [None] * 10 # Magic Number :(
    for digit in range(10):
        y_bin = (Y == digit).astype(float)
        thetas[digit] = (newton_method(X, y_bin, thetas[digit], iters))
    return thetas

def iterate(X, Y, batch_size=1000, iters=10):
    start = 0
    end = batch_size

    thetas = None
    i = 0
    while end < len(X):
        print(i)
        i += 1
        x_batch = X[start: end]
        y_batch = Y[start: end]
        thetas = multi_newton_method(x_batch, y_batch, thetas, iters)
        start += batch_size
        end += batch_size
    return thetas

thetas = iterate(x_train, y_train)
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58


In [19]:
# Make predictions

def predict(X, thetas):
    probs = np.array([Pr(X, theta) for theta in thetas])
    return np.argmax(probs, axis=0)

def accuracy(X, Y, thetas):
    preds = predict(X, thetas)
    return np.mean(preds==Y)

print(accuracy(x_test, y_test, thetas))


0.822
